###### setting work path

In [106]:
import os

print(os.getcwd())
os.chdir(r'C:\Users\dongwan.kim\Desktop\da_work\2000_trueskill')
print(os.getcwd())

C:\Users\dongwan.kim\Desktop\da_work\2000_trueskill
C:\Users\dongwan.kim\Desktop\da_work\2000_trueskill


###### loading file

In [107]:
import pandas as pd

file_name = '20180711_ratingSim.xlsx'
df = pd.read_excel(io=file_name, sheet_name='match_result', header=0)
df

,reg_date,match_id,team_no,player_id,result,weight
0,2018-07-01 11:00:00,1,0,7,0,1
1,2018-07-01 11:00:00,1,0,6,0,1
2,2018-07-01 11:00:00,1,1,3,1,1
3,2018-07-01 11:00:00,1,1,5,1,1
4,2018-07-01 11:00:00,2,0,6,0,1
5,2018-07-01 11:00:00,2,1,3,1,1


In [108]:
header_names = list(df)
header_names

['reg_date', 'match_id', 'team_no', 'player_id', 'result', 'weight']

###### trueskill settings

In [109]:
import trueskill as ts
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.stats import norm
pdf = norm.pdf
cdf = norm.cdf
icdf = norm.ppf  # inverse CDF

def get_draw_margin(p, beta, total_players=2):
    """ Compute the draw margin (epsilon) given the draw probability. """
    return icdf((p + 1.0) / 2) * math.sqrt(total_players) * beta

init_mu = 25
init_sigma = init_mu / 3
init_beta = init_sigma / 2
init_gamma = init_sigma / 100
init_draw_probability = 0.1
init_epsilon = get_draw_margin(0.1, init_beta); init_epsilon

env = ts.TrueSkill(mu=init_mu, sigma=init_sigma, beta=init_beta, tau=init_gamma, draw_probability=init_draw_probability, backend='scipy')

###### df transformer

In [110]:
import pandasql as pdsql


def get_transformed_df(df, col_name_reg_date, col_name_match_id):
    query = """
    select
        t2.match_min_reg_date
        , t1.*
    from 
        df as t1
        inner join
        (
            select 
                """ + col_name_match_id + """
                , min(""" + col_name_reg_date + """) as match_min_reg_date
            from 
                df
            group by 
                """ + col_name_match_id + """ 
        ) as t2
        on
            t1.""" + col_name_match_id + """ = t2.""" + col_name_match_id + """
    order by
        t2.match_min_reg_date
        , t1.""" + col_name_match_id + """
        , t1.result
    """
    return pdsql.sqldf(query, locals())   

In [111]:
# test code

import pandas as pd


file_name = '20180711_ratingSim.xlsx'
dfx = pd.read_excel(io=file_name, sheet_name='match_result', header=0)
get_transformed_df(dfx, 'reg_date', 'match_id')

,match_min_reg_date,reg_date,match_id,team_no,player_id,result,weight
0,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,1,0,7,0,1
1,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,1,0,6,0,1
2,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,1,1,3,1,1
3,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,1,1,5,1,1
4,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,2,0,6,0,1
5,2018-07-01 11:00:00.000000,2018-07-01 11:00:00.000000,2,1,3,1,1


###### slicing

In [112]:
def get_itermatch(df, col_name_reg_date, col_name_match_id, col_name_team_no):
    """
    df: Pandas dataframe

    yielding match : dict {
        result_code0: [player, ...]
        , result_code1: [player, ...]
        , ...
    }
    """

    def append_player(match, player):
        team_no = player[col_name_team_no]
        if match.get(team_no) is None:  # new team encountered
            match[team_no] = [player]
        else:
            match[team_no].append(player)

    last_match_id = None
    match = {}

    df = get_transformed_df(df, col_name_reg_date, col_name_match_id)

    for idx, row in df.iterrows():
        player = row.to_dict()
        this_match_id = player[col_name_match_id]

        if idx == 0 or last_match_id == this_match_id:  # 1st record or same match with last record
            append_player(match, player)
        else:  # new match record started
            yield match
            match = {}
            append_player(match, player)

        last_match_id = this_match_id

    yield match

In [128]:
# test code

file_name = '20180711_ratingSim.xlsx'
dfx = pd.read_excel(io=file_name, sheet_name='match_result', header=0)

itermatch = get_itermatch(dfx, 'reg_date', 'match_id', 'team_no')
list(itermatch)

[{0: [{'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 7,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'team_no': 0,
    'weight': 1},
   {'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 6,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'team_no': 0,
    'weight': 1}],
  1: [{'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 3,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 1,
    'team_no': 1,
    'weight': 1},
   {'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 5,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 1,
    'team_no': 1,
    'weight': 1}]},
 {0: [{'match_id': 2,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 6,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'team_no': 0,
    'weight': 1}],
  1: [{'match_id'

###### here we go

In [233]:
class RatingStore():
    def __init__(self, mmrtype: str, default_rating=None):
        self.store = dict()
        
        if mmrtype.lower() not in {'trueskill', 'elo'}:
            raise ValueError('Unknown mmrtype')
        else:
            self.mmrtype = mmrtype.lower()  # 'trueskill', 'elo', 'glicko', ...
        
        if default_rating is None:
            if self.mmrtype == 'trueskill':
                self.default_rating = {'score1':25, 'score2':8.333}
            elif self.mmrtype == 'elo':
                self.default_rating = {'score1':1200}
        
    def get_player_rating(self, player_id):
        # if it's new player then return default value, or return existing rating
        return self.store.get(player_id) or self.default_rating
        
    def update_rating(self, player_id, rating):
        if self.mmrtype == 'trueskill':
            _update_rating_trueskill(player_id, rating)
    
    def _update_rating_trueskill(self, player_id, rating):
        import trueskill
        assert isinstance(rating, trueskill.Rating)
        self.store[player_id] = {'score1':rating.mu, 'score2':rating.sigma}            

    def _update_rating_elo(self, player_id, rating):
        pass # not implemented
        
    

In [234]:
rating_store = RatingStore('trueskill')
rating_store.get_player_rating(1)

{'score1': 25, 'score2': 8.333}

In [235]:
# moved into update_match_result()

def get_rating_groups(match):
    rating_groups = []
    ranks = []

    for i, (result, team_members) in enumerate(match.items()):  # for each team(result) in the match
        rating_groups.append(dict())
        for j, player in enumerate(team_members):  # for each player in the team
            if j == 0:
                ranks.append(player['result'])

            playerid = player['player_id']

            r = rating_store.get_player_rating(player['player_id'])
            player['rating_before'] = env.Rating(r.get('scroe1'), r.get('score2'))
            rating_groups[i][playerid] = player['rating_before']
    
    return rating_groups, ranks

In [236]:
# test code

match = {0: [{'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 7,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'team_no': 0,
    'weight': 1},
   {'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 6,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'team_no': 0,
    'weight': 1}],
  1: [{'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 3,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 1,
    'team_no': 1,
    'weight': 1},
   {'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 5,
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 1,
    'team_no': 1,
    'weight': 1}]}

rating_groups_after, ranks = get_rating_groups(match)
rating_groups_after, ranks

([{6: trueskill.Rating(mu=25.000, sigma=8.333),
   7: trueskill.Rating(mu=25.000, sigma=8.333)},
  {3: trueskill.Rating(mu=25.000, sigma=8.333),
   5: trueskill.Rating(mu=25.000, sigma=8.333)}],
 [0, 1])

In [237]:
env.rate(rating_groups_after, ranks)

[{6: trueskill.Rating(mu=28.108, sigma=7.774),
  7: trueskill.Rating(mu=28.108, sigma=7.774)},
 {3: trueskill.Rating(mu=21.892, sigma=7.774),
  5: trueskill.Rating(mu=21.892, sigma=7.774)}]

In [238]:
# moved into update_match_result()
def set_rating_after(match, player_id, rating_after):
    for team_no, team_members in match.items():
        for i, player in enumerate(team_members):
            if player['player_id'] == player_id:
                match[team_no][i]['rating_after'] = rating_after
            else:
                pass

In [239]:
match

{0: [{'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 7,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 0,
   'team_no': 0,
   'weight': 1},
  {'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 6,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 0,
   'team_no': 0,
   'weight': 1}],
 1: [{'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 3,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 1,
   'team_no': 1,
   'weight': 1},
  {'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 5,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 1,
   'team_no': 1,
   'weight': 1

In [240]:
# test code

set_rating_after(match, 6, env.Rating(25, 3))

In [241]:
match

{0: [{'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 7,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 0,
   'team_no': 0,
   'weight': 1},
  {'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 6,
   'rating_after': trueskill.Rating(mu=25.000, sigma=3.000),
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 0,
   'team_no': 0,
   'weight': 1}],
 1: [{'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 3,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:00:00.000000',
   'result': 1,
   'team_no': 1,
   'weight': 1},
  {'match_id': 1,
   'match_min_reg_date': '2018-07-01 11:00:00.000000',
   'player_id': 5,
   'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
   'reg_date': '2018-07-01 11:0

In [248]:
def update_match_result(match, rating_groups_after):
    def get_rating_groups(match):
        rating_groups = []
        ranks = []

        for i, (result, team_members) in enumerate(match.items()):  # for each team(result) in the match
            rating_groups.append(dict())
            for j, player in enumerate(team_members):  # for each player in the team
                if j == 0:
                    ranks.append(player['result'])
                
                playerid = player['player_id']
                
                r = rating_store.get_player_rating(player['player_id'])
                player['rating_before'] = env.Rating(r.get('scroe1'), r.get('score2'))
                rating_groups[i][playerid] = player['rating_before']

        return rating_groups, ranks
    
    def set_rating_after(match, player_id, rating_after):
        for team_no, team_members in match.items():
            for i, player in enumerate(team_members):
                if player['player_id'] == player_id:
                    match[team_no][i]['rating_after'] = rating_after
                else:
                    pass
    
    rating_groups, ranks = get_rating_groups(match)
    rating_groups_after = env.rate(rating_groups, ranks)
    match_val_list = list(match.values())
    
    for i, rating_group in enumerate(rating_groups_after):
        for player_id, rating_after in rating_group.items():
            set_rating_after(match, player_id, rating_after)
            update_player_rating_store(player_id, rating_after)
            

In [249]:
itermatch = get_itermatch(dfx, 'reg_date', 'match_id', 'result')
match_history = list(itermatch)
for match in match_history:  # for each match(dict those key is teamno.)
    update_match_result(match, rating_groups_after)
    

In [250]:
match_history

[{0: [{'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 7,
    'rating_after': trueskill.Rating(mu=28.108, sigma=7.774),
    'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'team_no': 0,
    'weight': 1},
   {'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 6,
    'rating_after': trueskill.Rating(mu=28.108, sigma=7.774),
    'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 0,
    'team_no': 0,
    'weight': 1}],
  1: [{'match_id': 1,
    'match_min_reg_date': '2018-07-01 11:00:00.000000',
    'player_id': 3,
    'rating_after': trueskill.Rating(mu=21.892, sigma=7.774),
    'rating_before': trueskill.Rating(mu=25.000, sigma=8.333),
    'reg_date': '2018-07-01 11:00:00.000000',
    'result': 1,
    'team_no': 1,
    'weight': 1},
   {'match_id': 1,
    'match_min_re